In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np 
import pandas as pd 
import os

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
spacy_tok = spacy.load('en')
import glob

In [ ]:
df = pd.read_csv('../input/train.tsv',delimiter='\t')
df.head()

In [ ]:
df2= pd.read_csv('../input/test.tsv',delimiter='\t')
df2.head()

In [ ]:
#drop incomplete sentences in training set
def dropIncomplete (df, label="SentenceId", keepLocation=0):
    sentences = []
    for i in df[label].unique():  
        sentences.append(df[df[label] == i].reset_index(drop=True).loc[keepLocation])
    df_result = pd.DataFrame(sentences)
    return df_result

In [ ]:
df_train = dropIncomplete(df)
df_test = dropIncomplete(df2)

In [ ]:
df_train = pd.DataFrame(df_train["Phrase"]).reset_index(drop=True)
df_test = pd.DataFrame(df_test["Phrase"]).reset_index(drop=True)

In [ ]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [ ]:
bs=64; bptt=70

In [ ]:
PATH=''

In [ ]:
md = LanguageModelData.from_dataframes(path=PATH,field=TEXT,col="Phrase",train_df=df_train, val_df=df_test,
                                       test_df=df_test,bs=bs, bptt=bptt)

In [ ]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

In [ ]:
em_sz = 200
nh = 500
nl = 3

In [ ]:
opt_fn = partial(optim.Adam, betas = (0.7, 0.99))

In [ ]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05, dropout=0.05,
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)

In [ ]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [ ]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

In [ ]:
class MovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @staticmethod
    def sort_key(ex): return len(ex.text)
      
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [ ]:
MovieReviewLabel = data.Field(sequential=False)

In [ ]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [ ]:
trn = "train"
tst= "test"

In [ ]:
PATH = ''

In [ ]:
splits = MovieReview.splits(TEXT, MovieReviewLabel, PATH,train=trn,test=tst)
print(splits)

In [ ]:
bs=64; bptt=70

In [ ]:
md2 = TextData.from_splits(PATH, splits, bs)

In [ ]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_10_enc')

In [ ]:
accuracy_np(*m3.predict_with_targs())

In [ ]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [ ]:
m3.fit(lrs/2, 1, metrics=[accuracy])

In [ ]:
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

In [ ]:
m3.fit(.0001, 7, cycle_len=2,cycle_save_name='imdb2')